## [作業重點]
了解如何使用 Sklearn 中的 hyper-parameter search 找出最佳的超參數

### 作業
請使用不同的資料集，並使用 hyper-parameter search 的方式，看能不能找出最佳的超參數組合

In [1]:
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier

In [13]:
wine= datasets.load_wine()

x_train,x_test,y_train,y_test= train_test_split(wine.data, wine.target, test_size=0.2,  random_state=42)

clf= GradientBoostingClassifier()

In [14]:
clf.fit(x_train, y_train)
y_pred= clf.predict(x_test)
print(metrics.mean_squared_error(y_test, y_pred))


0.05555555555555555


In [15]:
#查詢內含超參數
from pprint import pprint
print('Parameters currently in use:\n')
pprint(clf.get_params())

Parameters currently in use:

{'criterion': 'friedman_mse',
 'init': None,
 'learning_rate': 0.1,
 'loss': 'deviance',
 'max_depth': 3,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_iter_no_change': None,
 'presort': 'auto',
 'random_state': None,
 'subsample': 1.0,
 'tol': 0.0001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}


In [66]:
random_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth}
pprint(random_grid)

{'max_depth': [10, 24, 38, 52, 66, 80, 94, 108, 122, 136, 150, None],
 'n_estimators': [100, 122, 144, 166, 188, 211, 233, 255, 277, 300]}


In [58]:
import numpy as np
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 300, num = 10)] #np.linspace 在指定的间隔内返回均匀间隔的数字。

max_depth = [int(x) for x in np.linspace(10, 150, num = 11)]
max_depth.append(None)

min_samples_leaf = [1, 2, 4]

param_grid = dict(n_estimators=n_estimators, max_depth=max_depth,  min_samples_leaf=min_samples_leaf)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

C:\Users\wayne\.conda\envs\py36\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   29.7s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   50.7s
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed:  1.1min finished
C:\Users\wayne\.conda\envs\py36\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


In [62]:
# 印出最佳結果與最佳參數
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.035211 using {'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 255}


In [63]:
grid_result.best_params_

{'max_depth': 10, 'min_samples_leaf': 2, 'n_estimators': 255}

In [64]:
# 使用最佳參數重新建立模型
clf_bestparam= GradientBoostingClassifier(n_estimators=grid_result.best_params_['n_estimators'],  #引用best_params_['']
                                          max_depth=grid_result.best_params_['max_depth'],
                                          min_samples_leaf = grid_result.best_params_['min_samples_leaf'] ,random_state=42)
# 訓練模型
clf_bestparam.fit(x_train, y_train)

# 預測測試集
y_pred = clf_bestparam.predict(x_test)

In [65]:
# 調整參數後約可降至 0.02777 的 MSE
print(metrics.mean_squared_error(y_test, y_pred))


0.027777777777777776
